In [5]:
import pandas as pd

df_RB = pd.read_csv("/kaggle/input/spatial-temporal/traffic_DLPRB.csv")
df_time = pd.read_csv("/kaggle/input/spatial-temporal/traffic_DLThpTime.csv")
df_number = pd.read_csv("/kaggle/input/spatial-temporal/traffic_MR_number.csv")
df = pd.read_csv("/kaggle/input/spatial-temporal/traffic_DLThpVol.csv")

sub = pd.read_csv("/kaggle/input/spatial-temporal/submission5.csv")

In [6]:
df_RB.rename(columns={'Unnamed: 0': 'hours'}, inplace=True)
df_time.rename(columns={'Unnamed: 0': 'hours'}, inplace=True)
df_number.rename(columns={'Unnamed: 0': 'hours'}, inplace=True)
df.rename(columns={'Unnamed: 0': 'hours'}, inplace=True)

# Creating Time features
df_RB['week'] = df_RB['hours'] // (7 * 24)
df_time['week'] = df_time['hours'] // (7 * 24)
df_number['week'] = df_number['hours'] // (7 * 24)
df['week'] = df['hours'] // (7 * 24)

df_RB['day_of_week'] = (df_RB['hours'] // 24) % 7
df_time['day_of_week'] = (df_time['hours'] // 24) % 7
df_number['day_of_week'] = (df_number['hours'] // 24) % 7
df['day_of_week'] = (df['hours'] // 24) % 7

df_RB['hour_of_week'] = df_RB['hours'] % 168
df_time['hour_of_week'] = df_time['hours'] % 168
df_number['hour_of_week'] = df_number['hours'] % 168
df['hour_of_week'] = df['hours'] % 168

def add_hour_of_day(dataframe):
    dataframe['hour_of_day'] = dataframe['hour_of_week'] % 24
    return dataframe

df_RB = add_hour_of_day(df_RB)
df_time = add_hour_of_day(df_time)
df_number = add_hour_of_day(df_number)
df = add_hour_of_day(df)

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Dropout, Input, MultiHeadAttention, LayerNormalization,
    GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint,
    TensorBoard
)
from datetime import datetime
import os
import gc
import psutil
import logging
from typing import Tuple, List, Dict

class Config:
    SEED = 42
    SEQUENCE_LENGTH = 72
    BATCH_SIZE = 16 
    EPOCHS = 200
    LEARNING_RATE = 0.001
    HEAD_SIZE = 128
    NUM_HEADS = 2
    FF_DIM = 2
    NUM_TRANSFORMER_BLOCKS = 2
    MLP_UNITS = [64, 32]
    DROPOUT = 0.3
    CHUNK_SIZE = 500  
    
    BASE_DIR = 'experiment_results'
    
    @classmethod
    def get_paths(cls, basestation: int, cell: int) -> Tuple[str, str, str, str]:
        exp_dir = os.path.join(cls.BASE_DIR, f'station_{basestation}_cell_{cell}')
        model_dir = os.path.join(exp_dir, 'models')
        log_dir = os.path.join(exp_dir, 'logs')
        pred_dir = os.path.join(exp_dir, 'predictions')
        
        for directory in [model_dir, log_dir, pred_dir]:
            os.makedirs(directory, exist_ok=True)
            
        return exp_dir, model_dir, log_dir, pred_dir

class MemoryManager:
    @staticmethod
    def get_memory_usage() -> float:
        """Return current memory usage in GB"""
        process = psutil.Process(os.getpid())
        return process.memory_info().rss / 1024 / 1024 / 1024
    
    @staticmethod
    def clear_memory() -> None:
        """Force garbage collection and clear tensorflow session"""
        gc.collect()
        tf.keras.backend.clear_session()

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, features: np.ndarray, targets: np.ndarray, batch_size: int, shuffle: bool = True):
        self.features = features
        self.targets = targets
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.features))
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self) -> int:
        return int(np.ceil(len(self.features) / self.batch_size))
    
    def __getitem__(self, idx: int) -> Tuple[np.ndarray, np.ndarray]:
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.indices))
        batch_indices = self.indices[start_idx:end_idx]
        
        # Load data in smaller chunks to reduce memory usage
        batch_features = np.empty((len(batch_indices),) + self.features.shape[1:])
        batch_targets = np.empty((len(batch_indices),) + self.targets.shape[1:])
        
        for i, idx in enumerate(batch_indices):
            batch_features[i] = self.features[idx]
            batch_targets[i] = self.targets[idx]
        
        return batch_features, batch_targets
    
    def on_epoch_end(self) -> None:
        if self.shuffle:
            np.random.shuffle(self.indices)

def create_model(input_shape: Tuple[int, int], output_size: int = 32) -> Model:
    inputs = Input(shape=input_shape)
    x = inputs
    
    for _ in range(Config.NUM_TRANSFORMER_BLOCKS):
        attention_output = MultiHeadAttention(
            num_heads=Config.NUM_HEADS,
            key_dim=Config.HEAD_SIZE,
            dropout=Config.DROPOUT
        )(x, x)
        x = LayerNormalization(epsilon=1e-6)(attention_output + x)
        
        ffn = Dense(Config.FF_DIM, activation="relu")(x)
        ffn = Dense(input_shape[-1])(ffn)
        x = LayerNormalization(epsilon=1e-6)(ffn + x)
    
    x = GlobalAveragePooling1D()(x)
    for units in Config.MLP_UNITS:
        x = Dense(units, activation="relu")(x)
        x = Dropout(Config.DROPOUT)(x)
    
    outputs = Dense(output_size, activation="linear")(x)
    return Model(inputs, outputs)

class DataProcessor:
    @staticmethod
    def process_cell_data(df: pd.DataFrame, df_RB: pd.DataFrame, 
                         df_time: pd.DataFrame, df_number: pd.DataFrame,
                         basestation: int, cell: int) -> Tuple[np.ndarray, np.ndarray]:
        cell_beams = [f"{basestation}_{cell}_{beam}" for beam in range(32)]
        
        # Process data in chunks to reduce memory usage
        chunk_size = Config.CHUNK_SIZE
        n_chunks = int(np.ceil(len(df) / chunk_size))
        
        features_list = []
        targets_list = []
        
        for i in range(n_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, len(df))
            
            chunk_slice = slice(start_idx, end_idx)
            
            # Process time features
            hour_data = df.iloc[chunk_slice]['hour_of_day'].values
            day_data = df.iloc[chunk_slice]['day_of_week'].values
            week_data = df.iloc[chunk_slice]['week'].values
            
            time_features = np.column_stack([
                np.sin(2 * np.pi * hour_data / 24),
                np.cos(2 * np.pi * hour_data / 24),
                np.sin(2 * np.pi * day_data / 7),
                np.cos(2 * np.pi * day_data / 7),
                np.sin(2 * np.pi * week_data / 52),
                np.cos(2 * np.pi * week_data / 52)
            ])
            
            # Process other features
            chunk_features = np.hstack([
                time_features,
                df_RB.iloc[chunk_slice][cell_beams].values,
                df_time.iloc[chunk_slice][cell_beams].values,
                df_number.iloc[chunk_slice][cell_beams].values
            ])
            
            chunk_targets = df.iloc[chunk_slice][cell_beams].values
            
            features_list.append(chunk_features)
            targets_list.append(chunk_targets)
            
            # Clear memory after each chunk
            MemoryManager.clear_memory()
        
        features = np.vstack(features_list)
        targets = np.vstack(targets_list)
        
        del features_list, targets_list
        MemoryManager.clear_memory()
        
        return np.nan_to_num(features, nan=0), np.nan_to_num(targets, nan=0)

class ModelTrainer:
    def __init__(self, basestation: int, cell: int):
        self.basestation = basestation
        self.cell = cell
        _, self.model_dir, self.log_dir, _ = Config.get_paths(basestation, cell)
    
    def train(self, features: np.ndarray, targets: np.ndarray) -> Tuple[Model, np.ndarray, int]:
        X, y = self._create_sequences(features, targets)
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=Config.SEED
        )
        
        # Clear memory after splitting
        del X, y
        MemoryManager.clear_memory()
        
        train_gen = DataGenerator(X_train, y_train, Config.BATCH_SIZE)
        val_gen = DataGenerator(X_val, y_val, Config.BATCH_SIZE, shuffle=False)
        
        model = create_model((Config.SEQUENCE_LENGTH, features.shape[1]))
        model.compile(
            optimizer=Adam(learning_rate=Config.LEARNING_RATE),
            loss='huber',
            metrics=['mae', 'mse']
        )
        
        callbacks = self._create_callbacks()
        
        model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=Config.EPOCHS,
            callbacks=callbacks,
            verbose=1
        )
        
        del train_gen, val_gen
        MemoryManager.clear_memory()
        
        return model, X_train[-1:], features.shape[1]
    
    @staticmethod
    def _create_sequences(features: np.ndarray, targets: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        X, y = [], []
        for i in range(len(features) - Config.SEQUENCE_LENGTH):
            X.append(features[i:(i + Config.SEQUENCE_LENGTH)])
            y.append(targets[i + Config.SEQUENCE_LENGTH])
            
            if i % Config.CHUNK_SIZE == 0:
                MemoryManager.clear_memory()
        
        return np.array(X), np.array(y)
    
    def _create_callbacks(self) -> List:
        return [
            EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6),
            ModelCheckpoint(
                filepath=os.path.join(self.model_dir, 'best_model.keras'),
                monitor='val_loss',
                save_best_only=True
            ),
            TensorBoard(log_dir=self.log_dir)
        ]

def run_pipeline(df: pd.DataFrame, df_RB: pd.DataFrame, 
                df_time: pd.DataFrame, df_number: pd.DataFrame) -> pd.DataFrame:
    logging.basicConfig(level=logging.INFO)
    
    # Configure GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    
    all_predictions = []
    
    for basestation in range(30):
        for cell in range(3):
            try:
                logging.info(f"Processing basestation {basestation}, cell {cell}")
                logging.info(f"Current memory usage: {MemoryManager.get_memory_usage():.2f} GB")
                
                # Process data
                features, targets = DataProcessor.process_cell_data(
                    df, df_RB, df_time, df_number,
                    basestation, cell
                )
                
                # Scale data
                feature_scaler = RobustScaler()
                target_scaler = RobustScaler()
                
                scaled_features = feature_scaler.fit_transform(features)
                scaled_targets = target_scaler.fit_transform(targets)
                
                # Train model
                trainer = ModelTrainer(basestation, cell)
                model, last_sequence, feature_dim = trainer.train(scaled_features, scaled_targets)
                
                # Generate predictions
                for week in [6, 11]:
                    predictions = generate_predictions_for_cell(
                        model, last_sequence,
                        feature_scaler, target_scaler,
                        basestation, cell, week,
                        feature_dim
                    )
                    all_predictions.extend(predictions)
                
                # Clear memory
                MemoryManager.clear_memory()
                
            except Exception as e:
                logging.error(f"Error processing basestation {basestation}, cell {cell}: {str(e)}")
                continue
    
    # Save predictions
    final_predictions = pd.DataFrame(all_predictions)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_predictions.to_csv(
        os.path.join(Config.BASE_DIR, f'all_predictions_{timestamp}.csv'),
        index=False
    )
    
    return final_predictions

if __name__ == "__main__":
    try:
        predictions_df = run_pipeline(df, df_RB, df_time, df_number)
        logging.info("Pipeline completed successfully!")
        
    except Exception as e:
        logging.error(f"Error in main execution: {str(e)}")

Epoch 1/200


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 7/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3680 - mae: 0.6840 - mse: 1.1774  

W0000 00:00:1729792392.608972     103 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


39/39 ━━━━━━━━━━━━━━━━━━━━ 26s 313ms/step - loss: 0.3801 - mae: 0.6925 - mse: 4.9286 - val_loss: 0.4243 - val_mae: 0.7453 - val_mse: 23.6891 - learning_rate: 0.0010
Epoch 2/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3433 - mae: 0.6622 - mse: 1.6349 - val_loss: 0.4209 - val_mae: 0.7485 - val_mse: 23.6719 - learning_rate: 0.0010
Epoch 3/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3771 - mae: 0.7036 - mse: 7.2698 - val_loss: 0.4195 - val_mae: 0.7451 - val_mse: 23.6731 - learning_rate: 0.0010
Epoch 4/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3732 - mae: 0.7008 - mse: 11.2952 - val_loss: 0.4182 - val_mae: 0.7464 - val_mse: 23.6618 - learning_rate: 0.0010
Epoch 5/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3822 - mae: 0.7121 - mse: 9.9993 - val_loss: 0.4172 - val_mae: 0.7476 - val_mse: 23.6617 - learning_rate: 0.0010
Epoch 6/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3514 - mae: 0.6836 - mse: 2.1691 - val_loss: 0.4165 - val_mae: 0.7513 - 

...

39/39 ━━━━━━━━━━━━━━━━━━━━ 24s 270ms/step - loss: 0.3126 - mae: 0.6323 - mse: 0.9122 - val_loss: 0.2902 - val_mae: 0.6081 - val_mse: 0.8141 - learning_rate: 0.0010
Epoch 2/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2833 - mae: 0.6037 - mse: 0.7977 - val_loss: 0.2864 - val_mae: 0.6079 - val_mse: 0.7999 - learning_rate: 0.0010
Epoch 3/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2816 - mae: 0.6049 - mse: 0.7927 - val_loss: 0.2835 - val_mae: 0.6103 - val_mse: 0.7868 - learning_rate: 0.0010
Epoch 4/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2765 - mae: 0.6026 - mse: 0.7872 - val_loss: 0.2843 - val_mae: 0.6095 - val_mse: 0.7901 - learning_rate: 0.0010
Epoch 5/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2690 - mae: 0.5964 - mse: 0.7310 - val_loss: 0.2830 - val_mae: 0.6130 - val_mse: 0.7814 - learning_rate: 0.0010
Epoch 6/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2770 - mae: 0.6082 - mse: 0.7433 - val_loss: 0.2827 - val_mae: 0.6113 - val_mse

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/39 ━━━━━━━━━━━━━━━━━━━━ 9:06 14s/step - loss: 0.3761 - mae: 0.7044 - mse: 1.0546

W0000 00:00:1729799979.606281     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


39/39 ━━━━━━━━━━━━━━━━━━━━ 25s 272ms/step - loss: 0.3387 - mae: 0.6401 - mse: 1.1246 - val_loss: 0.3314 - val_mae: 0.6372 - val_mse: 1.1220 - learning_rate: 0.0010
Epoch 2/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3134 - mae: 0.6197 - mse: 1.0463 - val_loss: 0.3195 - val_mae: 0.6439 - val_mse: 1.0693 - learning_rate: 0.0010
Epoch 3/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3143 - mae: 0.6350 - mse: 1.0749 - val_loss: 0.3171 - val_mae: 0.6437 - val_mse: 1.0596 - learning_rate: 0.0010
Epoch 4/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3120 - mae: 0.6360 - mse: 1.0537 - val_loss: 0.3172 - val_mae: 0.6443 - val_mse: 1.0607 - learning_rate: 0.0010
Epoch 5/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3021 - mae: 0.6295 - mse: 0.9360 - val_loss: 0.3135 - val_mae: 0.6503 - val_mse: 1.0383 - learning_rate: 0.0010
Epoch 6/200
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3178 - mae: 0.6496 - mse: 1.0842 - val_loss: 0.3123 - val_mae: 0.6530 - val_mse

In [14]:
submissions_df.to_csv('submission8.csv', index=False)
!mv submission6.csv/kaggle/working/

from IPython.display import FileLink
FileLink(r'submission8.csv')

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


mv: missing destination file operand after 'submission6.csv/kaggle/working/'
Try 'mv --help' for more information.


/kaggle/working/submission8.csv